# 던전 시스템 End-to-End Flow Test

실제 언리얼에서 보낸 JSON 구조로 전체 던전 생성 → raw_map 업데이트 → 에이전트 실행 → 밸런싱 저장 플로우를 테스트합니다.

## 플로우
1. 언리얼이 1층 raw_map 전달 → 1,2,3층 생성
2. 1층 보스방: Super Agent 실행 → balanced_map 생성
3. 1층 클리어: is_finishing=TRUE → balanced_map 반환
4. 언리얼이 2층 raw_map 전달 → DB 업데이트
5. 2층 보스방: Super Agent 실행 → balanced_map 생성
6. (동일하게 3층 진행)


In [1]:
import os, sys
from pathlib import Path


def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")


src_path = find_src_folder()
sys.path.append(str(src_path))

print(f"✅ src path: {src_path}")

✅ src path: C:\ProjectML\src


In [2]:
import json
import random
from typing import Dict, Any
from services.dungeon_service import get_dungeon_service, _normalize_room_keys


# 몬스터 좌표를 0-1 사이로 정규화하는 헬퍼
def normalize_monster_positions(raw_map: Dict[str, Any]) -> Dict[str, Any]:
    # 먼저 Unreal JSON을 정규화 (camelCase -> snake_case)
    normalized = _normalize_room_keys(raw_map)

    # 그 다음 좌표 정규화 (0-1 범위)
    for room in normalized.get("rooms", []):
        for monster in room.get("monsters", []):
            # 좌표가 1보다 크면 정규화
            if monster.get("pos_x", 0) > 1.0:
                monster["pos_x"] = round(random.uniform(0.1, 0.9), 2)
            if monster.get("pos_y", 0) > 1.0:
                monster["pos_y"] = round(random.uniform(0.1, 0.9), 2)
    return normalized


service = get_dungeon_service()
print("✅ DungeonService 초기화 완료")

✅ DungeonService 초기화 완료


In [3]:
# 실제 언리얼이 보낼 JSON 구조
unreal_raw_map = {
    "playerIds": [0],
    "heroineIds": [1],
    "rooms": [
        {
            "roomId": 0,
            "type": 0,
            "size": 4,
            "neighbors": [1],
            "monsters": [],
            "eventType": 0,
        },
        {
            "roomId": 1,
            "type": 2,
            "size": 4,
            "neighbors": [0, 2],
            "monsters": [],
            "eventType": 0,
        },
        {
            "roomId": 2,
            "type": 1,
            "size": 10,
            "neighbors": [3, 1],
            "monsters": [{"monsterId": 0, "posX": 0.5, "posY": 8.5}],
            "eventType": 1,
        },
        {
            "roomId": 3,
            "type": 1,
            "size": 12,
            "neighbors": [4, 2],
            "monsters": [{"monsterId": 0, "posX": 10.5, "posY": 14.5}],
            "eventType": 2,
        },
        {
            "roomId": 4,
            "type": 4,
            "size": 12,
            "neighbors": [3],
            "monsters": [],
            "eventType": 0,
        },
    ],
    "rewards": [],
}

print("✅ 언리얼 raw_map 준비 완료")
print(f"   - 플레이어: {unreal_raw_map['playerIds']}")
print(f"   - 영웅: {unreal_raw_map['heroineIds']}")
print(f"   - 방의 수: {len(unreal_raw_map['rooms'])}")

✅ 언리얼 raw_map 준비 완료
   - 플레이어: [0]
   - 영웅: [1]
   - 방의 수: 5


## Step 1: 던전 입장 (1층, 2층, 3층 생성)


In [4]:
print("=" * 80)
print("[Step 1] 던전 입장: 1층, 2층, 3층 생성")
print("=" * 80)

# 몬스터 좌표 정규화
normalized_raw_map = normalize_monster_positions(unreal_raw_map)

print("\n[몬스터 좌표 정규화 완료]")
for room in normalized_raw_map["rooms"]:
    if room.get("monsters"):
        print(f"  Room {room['room_id']}: ", end="")
        for m in room["monsters"]:
            print(
                f"Monster {m['monster_id']} at ({m['pos_x']:.2f}, {m['pos_y']:.2f})",
                end=" ",
            )
        print()

entrance_result = service.entrance(
    player_ids=[0], heroine_ids=[1], raw_map=normalized_raw_map
)

floor1_id = entrance_result["floor1_id"]
floor2_id = entrance_result["floor2_id"]
floor3_id = entrance_result["floor3_id"]

print(f"\n생성된 던전 ID:")
print(f"   Floor 1: {floor1_id}")
print(f"   Floor 2: {floor2_id}")
print(f"   Floor 3: {floor3_id}")

# Floor 1 상태 확인
print(f"\n[Floor 1 초기 상태]")
floor1_status = service.get_status(player_ids=[0])
print(f"   - Floor: {floor1_status['floor']}")
print(f"   - is_finishing: {floor1_status['is_finishing']}")
print(
    f"   - summary_info: {floor1_status['summary_info'][:50] if floor1_status['summary_info'] else '(비어있음)'}"
)

[Step 1] 던전 입장: 1층, 2층, 3층 생성

[몬스터 좌표 정규화 완료]
  Room 2: Monster 0 at (0.50, 0.11) 
  Room 3: Monster 0 at (0.56, 0.16) 


OperationalError: (psycopg2.OperationalError) connection to server at "aws-1-ap-northeast-2.pooler.supabase.com" (3.39.47.126), port 5432 failed: FATAL:  MaxClientsInSessionMode: max clients reached - in Session mode max clients are limited to pool_size

(Background on this error at: https://sqlalche.me/e/20/e3q8)

## Step 2: 1층 보스방 - 에이전트 실행 및 밸런싱 저장

Super Dungeon Agent가 raw_map을 분석하여 balanced_map과 summary_info를 생성합니다.


In [5]:
print("\n" + "=" * 80)
print("[Step 2] 1층 보스방: Super Agent 실행")
print("=" * 80)

# Monster DB 가져오기
from agents.dungeon.monster.monster_database import MONSTER_DATABASE

# Super Agent 실행 (실제)
print("\n[Super Agent 실행 중...]")
balance_result = service.balance_dungeon(
    dungeon_id=floor1_id,
    heroine_data={
        "heroine_id": "1",
        "name": "레티아",
        "event_room": 3,
        "memory_progress": 50,
    },
    heroine_stat={
        "hp": 500,
        "strength": 20,
        "dexterity": 15,
        "intelligence": 10,
        "defense": 10,
        "critChance": 15.0,
        "attackSpeed": 1.5,
        "moveSpeed": 400,
        "skillDamageMultiplier": 1.2,
    },
    heroine_memories=[],
    dungeon_player_data={
        "affection": 50,
        "sanity": 80,
        "difficulty_level": "normal",
    },
    monster_db=MONSTER_DATABASE,
    used_events=[],
)

if balance_result["success"]:
    print(f"\n✅ 밸런싱 완료!")

    agent_result = balance_result["agent_result"]
    monster_stats = agent_result.get("monster_stats", {})
    difficulty_info = agent_result.get("difficulty_info", {})
    events = agent_result.get("events", {})

    print(f"\n[에이전트 결과 요약]")
    print(f"  - 몬스터 배치: {monster_stats.get('total_count', 0)}마리")
    print(f"  - 위협도 달성률: {monster_stats.get('achievement_rate', 0):.1f}%")
    print(f"  - AI 배율: x{difficulty_info.get('ai_multiplier', 1.0):.2f}")
    print(f"  - 메인 이벤트: {events.get('main_event', 'N/A')[:60]}...")

    print(f"\n[생성된 Summary Info]")
    print(
        balance_result["summary"][:300] if balance_result["summary"] else "(비어있음)"
    )
else:
    print(f"\n밸런싱 실패: {balance_result.get('error', 'Unknown error')}")


[Step 2] 1층 보스방: Super Agent 실행

[Super Agent 실행 중...]

[Dungeon 1] Super Agent 실행 중...

[Event Node] 이벤트 생성 시작...

[Monster Node] 몬스터 밸런싱 시작...
[calculate_combat_score_node] 플레이어 전투력: 194.00
[calculate_combat_score_node] HP: 500, STR: 20, DEX: 15
[llm_strategy_node WARNING] hero_summary 확인 불가
[heroine_memories_node] 히로인 ID: 1
[heroine_memories_node] 기억 진척도: 50
[heroine_memories_node] 해금된 기억 개수: 2
[selected_main_event_node] 선택된 이벤트: 심연을 숭배하는 자
[selected_main_event_node] 개별 이벤트 여부: False
[create_sub_event_node] 서브 이벤트 생성 완료
sub_event_narrative='축축한 돌바닥 위, 방 한가운데에 후드를 깊이 눌러쓴 인물이 굳어 서 있다. 헐렁한 천옷이 미세하게 떨릴 때마다, 그 안에서 낮게 갈라진 숨소리가 새어나온다. 당신을 눈치챈 듯 고개가 느리게 돌아가더니, 입술이 경련하듯 움직인다. "그, 그으… 심, 심연이… 하, 한…" 단어의 껍데기만 이어질 뿐, 문장은 곧 이상한 음절로 뒤엉켜 알아들을 수 없는 잡음이 된다. 공기 자체가 눌리는 듯한 압박감이 방을 채우고, 인물의 손끝은 떨리며 허공에 의미 모를 곡선을 그려낸다. 그 떨림 속에, 이해할 수 없는 무엇인가가 꿈틀거리는 기분이 든다.' event_choices=[EventChoice(action='조심스럽게 다가가 인물의 말을 따라 해 보며 소통을 시도한다', reward_id='cooldown_reduction_all', penalty_id=None), EventChoice(action='인물

## Step 3: 1층 클리어 - 완료 처리 및 balanced_map 반환

is_finishing=TRUE로 설정하고, Unreal이 다음 층 raw_map 생성에 사용할 balanced_map을 반환합니다.


In [6]:
print("\n" + "=" * 80)
print("[Step 3] 1층 클리어: 완료 처리")
print("=" * 80)

finish_result = service.clear_floor(player_ids=[0])

if finish_result["success"]:
    print(f"\n✅ 1층 완료 처리 성공")

    finished_dungeon = finish_result["finished_dungeon"]
    balanced_map = finish_result["balanced_map"]

    print(f"\n[완료 처리된 던전 정보]")
    print(f"  - Floor: {finished_dungeon['floor']}")
    print(f"  - is_finishing: {finished_dungeon['is_finishing']}")
    print(f"  - summary_info (첫 200자):")
    print(
        f"    {finished_dungeon['summary_info'][:200] if finished_dungeon['summary_info'] else '(비어있음)'}"
    )

    print(f"\n[Unreal에 반환할 balanced_map]")
    print(f"  - playerIds: {balanced_map.get('playerIds', [])}")
    print(f"  - heroineIds: {balanced_map.get('heroineIds', [])}")
    print(f"  - 방의 수: {len(balanced_map.get('rooms', []))}")

    # 다음 층을 위해 balanced_map 저장
    floor1_balanced_map = balanced_map
else:
    print(f"{finish_result.get('error', 'Unknown error')}")


[Step 3] 1층 클리어: 완료 처리

✅ 1층 완료 처리 성공

[완료 처리된 던전 정보]
  - Floor: 1
  - is_finishing: True
  - summary_info (첫 200자):
    [맵 구성]
  - 총 방: 5개 (boss실 1개, empty실 1개, event실 1개, monster실 2개)

[몬스터 배치]
  - 총 4마리 (보스: 1, 일반: 3)
  - 위협도 달성률: 1406.1%

[난이도]
  - 전투력: 194.00
  - AI 배율: x0.90

[이벤트]
  - 주요: [심연을 숭배하는 자]
Event Code:

[Unreal에 반환할 balanced_map]
  - playerIds: [0]
  - heroineIds: [1]
  - 방의 수: 5


## Step 4: 2층 raw_map 업데이트

Unreal이 1층 balanced_map을 기반으로 생성한 2층 raw_map을 Server에 전달합니다.


In [7]:
print("\n" + "=" * 80)
print("[Step 4] 2층 raw_map 업데이트")
print("=" * 80)

# Unreal이 1층 balanced_map을 기반으로 생성한 2층 raw_map (시뮬레이션)
unreal_raw_map_floor2 = {
    "playerIds": [0],
    "heroineIds": [1],
    "rooms": [
        {
            "roomId": 0,
            "type": 0,  # 빈방
            "size": 6,
            "neighbors": [1],
            "monsters": [],
            "eventType": 0,
        },
        {
            "roomId": 1,
            "type": 1,  # 전투방
            "size": 14,
            "neighbors": [0, 2],
            "monsters": [
                {"monsterId": 4, "posX": 0.22, "posY": 0.40},
                {"monsterId": 5, "posX": 0.63, "posY": 0.78},
            ],
            "eventType": 0,
        },
        {
            "roomId": 2,
            "type": 4,  # 보스방
            "size": 20,
            "neighbors": [1],
            "monsters": [
                {"monsterId": 99, "posX": 0.5, "posY": 0.5},  # 보스 몬스터 1마리
            ],
            "eventType": 0,
        },
    ],
    "rewards": [201, 202],
}

# 좌표 정규화
normalized_floor2 = normalize_monster_positions(unreal_raw_map_floor2)

# 2층 업데이트
update_success = service.update_raw_map(floor2_id, normalized_floor2)

if update_success:
    print(f"2층 raw_map 업데이트 완료 (ID: {floor2_id})")
    print(f"   - 방의 수: {len(normalized_floor2['rooms'])}")
else:
    print(f"2층 raw_map 업데이트 실패")

# 2층 상태 확인
floor2_status = service.get_status(player_ids=[0])
if floor2_status and floor2_status["floor"] == 2:
    print(f"\n[2층 입장 직후 상태]")
    print(f"   - Floor: {floor2_status['floor']}")
    print(f"   - is_finishing: {floor2_status['is_finishing']}")
    print(
        f"   - balanced_map: {'있음' if floor2_status['balanced_map'] else '없음 (NULL)'}"
    )


[Step 4] 2층 raw_map 업데이트
2층 raw_map 업데이트 완료 (ID: 2)
   - 방의 수: 3

[2층 입장 직후 상태]
   - Floor: 2
   - is_finishing: False
   - balanced_map: 있음


## Step 5: 2층 보스방 - 에이전트 실행 및 밸런싱 저장


In [ ]:
print("\n" + "=" * 80)
print("[DEBUG] Step 4 후: 2층 raw_map이 제대로 업데이트되었는지 확인")
print("=" * 80)

# DB에서 직접 2층 raw_map 조회
from sqlalchemy import text
from db.RDBRepository import RDBRepository

repo = RDBRepository()
with repo.engine.connect() as conn:
    result = conn.execute(
        text("SELECT id, floor, raw_map FROM dungeon WHERE id = :id"),
        {"id": floor2_id}
    ).fetchone()
    
    if result:
        dungeon_id, floor, raw_map_str = result
        raw_map_data = json.loads(raw_map_str) if isinstance(raw_map_str, str) else raw_map_str
        
        print(f"\n✅ 2층 (ID: {dungeon_id}, Floor: {floor}) raw_map 확인:")
        print(f"   - rooms 개수: {len(raw_map_data.get('rooms', []))}")
        print(f"   - player_ids: {raw_map_data.get('player_ids')}")
        
        print(f"\n[각 room 상세:]")
        for i, room in enumerate(raw_map_data.get('rooms', [])):
            monsters = room.get('monsters', [])
            print(f"   Room {i}: type={room.get('room_type')}, size={room.get('size')}, monsters={monsters}")
        
        print(f"\n[예상값 (Step 4에서 업로드한 normalized_floor2)]")
        print(f"   - rooms 개수: 3 (room 0, 1, 2만)")
        print(f"   - room 1에 monsters: [4, 5] (monster ID)")
        
        # 3개 room 확인
        if len(raw_map_data.get('rooms', [])) == 3:
            print(f"\n✅ 올바름! raw_map이 제대로 업데이트됨")
        else:
            print(f"\n❌ 잘못됨! raw_map이 업데이트 안 됨 (여전히 1층 데이터)")
    else:
        print(f"❌ 던전 {floor2_id}을 찾을 수 없습니다")


In [8]:
print("\n" + "=" * 80)
print("[Step 5] 2층 보스방: Super Agent 실행")
print("=" * 80)

# 2층 밸런싱 실행
print("\n[Super Agent 실행 중...]")
balance_result_floor2 = service.balance_dungeon(
    dungeon_id=floor2_id,
    heroine_data={
        "heroine_id": "1",
        "name": "레티아",
        "event_room": 1,
        "memory_progress": 80,
    },
    heroine_stat={
        "hp": 550,
        "strength": 22,
        "dexterity": 16,
        "intelligence": 12,
        "defense": 12,
        "critChance": 17.0,
        "attackSpeed": 1.6,
        "moveSpeed": 420,
        "skillDamageMultiplier": 1.3,
    },
    heroine_memories=[],
    dungeon_player_data={
        "affection": 55,
        "sanity": 75,
        "difficulty_level": "normal",
    },
    monster_db=MONSTER_DATABASE,
    used_events=[],
)

if balance_result_floor2["success"]:
    print(f"\n2층 밸런싱 완료!")

    agent_result_floor2 = balance_result_floor2["agent_result"]
    monster_stats = agent_result_floor2.get("monster_stats", {})
    difficulty_info = agent_result_floor2.get("difficulty_info", {})

    print(f"\n[에이전트 결과 요약]")
    print(f"  - 몬스터 배치: {monster_stats.get('total_count', 0)}마리")
    print(f"  - 위협도 달성률: {monster_stats.get('achievement_rate', 0):.1f}%")
    print(f"  - AI 배율: x{difficulty_info.get('ai_multiplier', 1.0):.2f}")

    print(f"\n[생성된 Summary Info]")
    print(
        balance_result_floor2["summary"][:300]
        if balance_result_floor2["summary"]
        else "(비어있음)"
    )
else:
    print(f"\n2층 밸런싱 실패: {balance_result_floor2.get('error', 'Unknown error')}")


[Step 5] 2층 보스방: Super Agent 실행

[Super Agent 실행 중...]

[Dungeon 2] Super Agent 실행 중...

[Event Node] 이벤트 생성 시작...

[Monster Node] 몬스터 밸런싱 시작...
[calculate_combat_score_node] 플레이어 전투력: 212.40
[calculate_combat_score_node] HP: 550, STR: 22, DEX: 16
[heroine_memories_node] 히로인 ID: 1
[heroine_memories_node] 기억 진척도: 80
[heroine_memories_node] 해금된 기억 개수: 5
[selected_main_event_node] 선택된 이벤트: 검은 형상의 무언가
[selected_main_event_node] 개별 이벤트 여부: False
[llm_strategy_node WARNING] hero_summary 확인 불가

[select_monsters_node] 타겟 위협도: 233.64
[select_monsters_node] 배율: 1.10
[select_monsters_node] 선호도 조건: 3개
[select_monsters_node] 회피 조건: ['HighAttack', 'VeryFast']
[_select_monsters_by_strategy] 타겟: 233.64, 필터된 몬스터 수: 2
[보스방] 방 2: 던전 보스 배치
[전투방] 방 1: 2마리 배치
[select_monsters_node] 일반 몬스터 수: 4
[select_monsters_node] 보스방 존재: True
[select_monsters_node] 일반 몬스터 위협도: 255.00
[select_monsters_node] 보스 위협도: 2250.00
[select_monsters_node] 총 위협도: 2505.00
[select_monsters_node] 달성률: 1072.2%
[Monster Node] 완료:
  - 전투

## 최종 검증: 전체 플로우 확인

모든 던전 데이터를 조회하여 완전한 플로우가 작동했는지 검증합니다.


## Step 6: 2층 클리어 - 완료 처리 및 3층 진입

In [9]:
print("\n" + "=" * 80)
print("[Step 6] 2층 클리어: 완료 처리")
print("=" * 80)

finish_result_floor2 = service.clear_floor(player_ids=[0])

if finish_result_floor2["success"]:
    print(f"\n✅ 2층 완료 처리 성공")

    finished_dungeon_floor2 = finish_result_floor2["finished_dungeon"]
    balanced_map_floor2 = finish_result_floor2["balanced_map"]

    print(f"\n[완료 처리된 던전 정보]")
    print(f"  - Floor: {finished_dungeon_floor2['floor']}")
    print(f"  - is_finishing: {finished_dungeon_floor2['is_finishing']}")
    print(f"  - summary_info (첫 200자):")
    print(
        f"    {finished_dungeon_floor2['summary_info'][:200] if finished_dungeon_floor2['summary_info'] else '(비어있음)'}"
    )

    print(f"\n[Unreal에 반환할 balanced_map]")
    print(f"  - playerIds: {balanced_map_floor2.get('playerIds', [])}")
    print(f"  - heroineIds: {balanced_map_floor2.get('heroineIds', [])}")
    print(f"  - 방의 수: {len(balanced_map_floor2.get('rooms', []))}")

    # 다음 층을 위해 balanced_map 저장
    floor2_balanced_map = balanced_map_floor2
else:
    print(f"{finish_result_floor2.get('error', 'Unknown error')}")


[Step 6] 2층 클리어: 완료 처리

✅ 2층 완료 처리 성공

[완료 처리된 던전 정보]
  - Floor: 2
  - is_finishing: True
  - summary_info (첫 200자):
    [맵 구성]
  - 총 방: 3개 (boss실 1개, empty실 1개, monster실 1개)

[몬스터 배치]
  - 총 3마리 (보스: 1, 일반: 2)
  - 위협도 달성률: 1072.2%

[난이도]
  - 전투력: 212.40
  - AI 배율: x1.10

[이벤트]
  - 주요: [검은 형상의 무언가]
Event Code: BLACK_FIGU

[Unreal에 반환할 balanced_map]
  - playerIds: []
  - heroineIds: []
  - 방의 수: 5


## Step 7: 3층 raw_map 업데이트

In [10]:
print("\n" + "=" * 80)
print("[Step 7] 3층 raw_map 업데이트")
print("=" * 80)

# Unreal이 2층 balanced_map을 기반으로 생성한 3층 raw_map (시뮬레이션)
unreal_raw_map_floor3 = {
    "playerIds": [0],
    "heroineIds": [1],
    "rooms": [
        {
            "roomId": 0,
            "type": 0,  # 빈방
            "size": 8,
            "neighbors": [1],
            "monsters": [],
            "eventType": 0,
        },
        {
            "roomId": 1,
            "type": 2,  # 이벤트방
            "size": 10,
            "neighbors": [0, 2],
            "monsters": [],
            "eventType": 3,
        },
        {
            "roomId": 2,
            "type": 1,  # 전투방
            "size": 16,
            "neighbors": [1, 3],
            "monsters": [
                {"monsterId": 7, "posX": 0.15, "posY": 0.25},
                {"monsterId": 8, "posX": 0.45, "posY": 0.55},
                {"monsterId": 9, "posX": 0.75, "posY": 0.85},
            ],
            "eventType": 0,
        },
        {
            "roomId": 3,
            "type": 1,  # 전투방
            "size": 18,
            "neighbors": [2, 4],
            "monsters": [
                {"monsterId": 10, "posX": 0.30, "posY": 0.40},
                {"monsterId": 11, "posX": 0.70, "posY": 0.60},
            ],
            "eventType": 0,
        },
        {
            "roomId": 4,
            "type": 3,  # 보물방
            "size": 12,
            "neighbors": [3, 5],
            "monsters": [],
            "eventType": 0,
        },
        {
            "roomId": 5,
            "type": 4,  # 보스방 (최종)
            "size": 25,
            "neighbors": [4],
            "monsters": [
                {"monsterId": 99, "posX": 0.5, "posY": 0.5},  # 최종 보스
            ],
            "eventType": 0,
        },
    ],
    "rewards": [301, 302, 303],
}

# 좌표 정규화
normalized_floor3 = normalize_monster_positions(unreal_raw_map_floor3)

# 3층 업데이트
update_success_floor3 = service.update_raw_map(floor3_id, normalized_floor3)

if update_success_floor3:
    print(f"3층 raw_map 업데이트 완료 (ID: {floor3_id})")
    print(f"   - 방의 수: {len(normalized_floor3['rooms'])}")
else:
    print(f"3층 raw_map 업데이트 실패")

# 3층 상태 확인
floor3_status = service.get_status(player_ids=[0])
if floor3_status and floor3_status["floor"] == 3:
    print(f"\n[3층 입장 직후 상태]")
    print(f"   - Floor: {floor3_status['floor']}")
    print(f"   - is_finishing: {floor3_status['is_finishing']}")
    print(
        f"   - balanced_map: {'있음' if floor3_status['balanced_map'] else '없음 (NULL)'}"
    )


[Step 7] 3층 raw_map 업데이트
3층 raw_map 업데이트 완료 (ID: 3)
   - 방의 수: 6

[3층 입장 직후 상태]
   - Floor: 3
   - is_finishing: False
   - balanced_map: 있음


## Step 8: 3층 보스방 - 최종 밸런싱

In [11]:
print("\n" + "=" * 80)
print("[Step 8] 3층 보스방: Super Agent 실행 (최종)")
print("=" * 80)

# 3층 밸런싱 실행 (최종 층)
print("\n[Super Agent 실행 중...]")
balance_result_floor3 = service.balance_dungeon(
    dungeon_id=floor3_id,
    heroine_data={
        "heroine_id": "1",
        "name": "레티아",
        "event_room": 2,
        "memory_progress": 100,
    },
    heroine_stat={
        "hp": 600,
        "strength": 25,
        "dexterity": 18,
        "intelligence": 15,
        "defense": 15,
        "critChance": 20.0,
        "attackSpeed": 1.7,
        "moveSpeed": 450,
        "skillDamageMultiplier": 1.5,
    },
    heroine_memories=[],
    dungeon_player_data={
        "affection": 70,
        "sanity": 60,
        "difficulty_level": "hard",
    },
    monster_db=MONSTER_DATABASE,
    used_events=[],
)

if balance_result_floor3["success"]:
    print(f"\n✅ 3층 밸런싱 완료!")

    agent_result_floor3 = balance_result_floor3["agent_result"]
    monster_stats = agent_result_floor3.get("monster_stats", {})
    difficulty_info = agent_result_floor3.get("difficulty_info", {})
    events = agent_result_floor3.get("events", {})

    print(f"\n[에이전트 결과 요약]")
    print(f"  - 몬스터 배치: {monster_stats.get('total_count', 0)}마리")
    print(f"  - 위협도 달성률: {monster_stats.get('achievement_rate', 0):.1f}%")
    print(f"  - AI 배율: x{difficulty_info.get('ai_multiplier', 1.0):.2f}")
    print(f"  - 메인 이벤트: {events.get('main_event', 'N/A')[:60]}...")

    print(f"\n[생성된 Summary Info]")
    print(
        balance_result_floor3["summary"][:300]
        if balance_result_floor3["summary"]
        else "(비어있음)"
    )
else:
    print(
        f"\n❌ 3층 밸런싱 실패: {balance_result_floor3.get('error', 'Unknown error')}"
    )


[Step 8] 3층 보스방: Super Agent 실행 (최종)

[Super Agent 실행 중...]

[Dungeon 3] Super Agent 실행 중...

[Event Node] 이벤트 생성 시작...

[Monster Node] 몬스터 밸런싱 시작...
[calculate_combat_score_node] 플레이어 전투력: 232.20
[calculate_combat_score_node] HP: 600, STR: 25, DEX: 18
[heroine_memories_node] 히로인 ID: 1
[heroine_memories_node] 기억 진척도: 100
[heroine_memories_node] 해금된 기억 개수: 6
[selected_main_event_node] 선택된 이벤트: 쓰러져있는 사람
[selected_main_event_node] 개별 이벤트 여부: False
[llm_strategy_node WARNING] hero_summary 확인 불가
[create_sub_event_node] 서브 이벤트 생성 완료
sub_event_narrative='습기 찬 돌바닥 위, 방 구석에 한 사람이 몸을 웅크린 채 쓰러져 있다. 너덜너덜해진 후드 망토가 전신을 뒤덮어 얼굴은 완전히 그림자 속에 잠겨 있다. 옅은 숨소리가 새어 나오지만, 규칙적이지 않고 위태롭다. 주변에는 발자국이나 피자국 하나 없이, 마치 이 방 자체가 그를 삼켜 버린 듯한 고요만 감돌 뿐이다. 곰팡이 냄새와 메마른 숨소리가 뒤섞인 가운데, 후드 틈새에서 희미한 한기가 퍼져 나온다. 이 존재가 구조를 기다리는 자인지, 함정을 품은 미지의 위협인지는 아직 알 수 없다.' event_choices=[EventChoice(action='쓰러진 사람에게 다가가 상태를 살피고 응급처치를 시도한다.', reward_id='hp_recovery_all', penalty_id=None), EventChoice(action='거리를 둔 채 무기를 겨누고, 정체를 확인할 때까지 날카롭게 말

## Step 9: 3층 클리어 - 던전 완전 종료

In [12]:
print("\n" + "=" * 80)
print("[Step 9] 3층 클리어: 던전 완전 종료")
print("=" * 80)

finish_result_floor3 = service.clear_floor(player_ids=[0])

if finish_result_floor3["success"]:
    print(f"\n✅ 3층 완료 처리 성공 - 던전 완전 클리어!")

    finished_dungeon_floor3 = finish_result_floor3["finished_dungeon"]
    balanced_map_floor3 = finish_result_floor3["balanced_map"]

    print(f"\n[완료 처리된 던전 정보]")
    print(f"  - Floor: {finished_dungeon_floor3['floor']}")
    print(f"  - is_finishing: {finished_dungeon_floor3['is_finishing']}")
    print(f"  - summary_info (첫 200자):")
    print(
        f"    {finished_dungeon_floor3['summary_info'][:200] if finished_dungeon_floor3['summary_info'] else '(비어있음)'}"
    )

    print(f"\n[최종 balanced_map]")
    print(f"  - playerIds: {balanced_map_floor3.get('playerIds', [])}")
    print(f"  - heroineIds: {balanced_map_floor3.get('heroineIds', [])}")
    print(f"  - 방의 수: {len(balanced_map_floor3.get('rooms', []))}")

    print(f"\n모든 층 완료! 던전 정복 성공!")
else:
    print(f"{finish_result_floor3.get('error', 'Unknown error')}")


[Step 9] 3층 클리어: 던전 완전 종료

✅ 3층 완료 처리 성공 - 던전 완전 클리어!

[완료 처리된 던전 정보]
  - Floor: 3
  - is_finishing: True
  - summary_info (첫 200자):
    [맵 구성]
  - 총 방: 6개 (boss실 1개, empty실 1개, event실 1개, monster실 2개, treasure실 1개)

[몬스터 배치]
  - 총 5마리 (보스: 1, 일반: 4)
  - 위협도 달성률: 1027.4%

[난이도]
  - 전투력: 232.20
  - AI 배율: x1.05

[이벤트]
  - 주요: [쓰러져있는 사람]

[최종 balanced_map]
  - playerIds: []
  - heroineIds: []
  - 방의 수: 3

모든 층 완료! 던전 정복 성공!
